# PSK II. Makró blokk beadandó

### Szabályok

  - Lehet (de nem kötelező) maximum három fós csoportokban dolgozni
  - A megoldásokat küldjétek el a martin.stancsics@gmail.com email címre a szakmai szabályzatban meghatározott határidőig

### Instrukciók

  - TODO

In [1]:
import dolo

## 1. feladat: Egy kalibrált RBC modell

Választott ország: #############

In [2]:
model_baseline = dolo.yaml_import("rbc_baseline.yaml")
print(model_baseline)


        Model:
        ------
        name: "None"
        type: "dtcc"
        file: "<string>

Equations:
----------

transition
 1  : 0.0000 : a[t] = exp((rho)*(log(a[t-1])) + eps[t])
 2  : 0.0000 : k[t] = (1 - (delta))*(k[t-1]) + i[t-1]

arbitrage
 1  : 0.1462 : (-(1))/(c[t]) + ((beta)*(1 + r[t+1] - (delta)))*((1)/(c[t+1]))
 2  : 5.2169 : -(w[t]) + ((phi)*(c[t]))/(1 - (l[t]))

definitions
 1  : y[t] = ((exp(a[t]))*((k[t])^(alpha)))*((l[t])^(1 - (alpha)))
 2  : c[t] = y[t] - (i[t])
 3  : r[t] = ((alpha)*(y[t]))/(k[t])
 4  : w[t] = ((1 - (alpha))*(y[t]))/(l[t])




### Steady state

In [3]:
ss_baseline = dolo.algos.steady_state.find_steady_state(model_baseline)
model_baseline.set_calibration(**ss_baseline.flat)

Ellenőrizd, hogy valóban a steady-statet találtad-e meg

In [4]:
print(model_baseline)


        Model:
        ------
        name: "None"
        type: "dtcc"
        file: "<string>

Equations:
----------

transition
 1  : 0.0000 : a[t] = exp((rho)*(log(a[t-1])) + eps[t])
 2  : 0.0000 : k[t] = (1 - (delta))*(k[t-1]) + i[t-1]

arbitrage
 1  : 0.0000 : (-(1))/(c[t]) + ((beta)*(1 + r[t+1] - (delta)))*((1)/(c[t+1]))
 2  : 0.0000 : -(w[t]) + ((phi)*(c[t]))/(1 - (l[t]))

definitions
 1  : y[t] = ((exp(a[t]))*((k[t])^(alpha)))*((l[t])^(1 - (alpha)))
 2  : c[t] = y[t] - (i[t])
 3  : r[t] = ((alpha)*(y[t]))/(k[t])
 4  : w[t] = ((1 - (alpha))*(y[t]))/(l[t])




In [5]:
model_baseline.get_calibration()

{'beta': 0.94,
 'phi': 4.0,
 'delta': 0.06,
 'alpha': 0.33,
 'rho': 0.65,
 'l': 0.16623609157181168,
 'a': 0.9999999999999999,
 'r': 0.12382978723404253,
 'k': 3.193614388771363,
 'i': 0.19161686332628192,
 'c': 0.2492415987079727,
 'w': 1.7768413993019831,
 'name': 0.0,
 'y': 1.1983775462701163,
 'eps': 0}

### Szimuláció

In [6]:
A modell megoldása:

dr_baseline = dolo.approximate_controls(model_baseline)

Szimulálj 1000 időszaknyi adatot

In [7]:
sim_baseline = dolo.simulate(model_baseline, dr_baseline, N=1, T=1000).squeeze()
sim_df_baseline = sim_baseline \
    .to_dataframe(name="value") \
    .drop(columns="N") \
    .reset_index() \
    .pivot(index="T", columns="V") \
    ["value"]
sim_df_baseline.head()

V,a,c,eps,i,k,l,r,w,y
T,,,,,,,,,
0,1.000000,1.006761,0.090114,0.191617,3.193614,0.166236,0.123830,4.829956,1.198378
1,1.094299,1.062601,-0.069685,0.337279,3.193614,0.182114,0.144651,5.150187,1.399881
2,0.988950,1.023814,0.040456,0.158351,3.339277,0.162001,0.116826,4.889167,1.182165
3,1.033792,1.040528,-0.127744,0.232288,3.297271,0.170236,0.127387,5.009437,1.272816
4,0.899297,0.991340,-0.027555,0.020705,3.331723,0.147029,0.100241,4.611821,1.012045


### Kiértékelés

Átlaghoz viszonyított szórások:

In [8]:
sim_df_baseline.std() / sim_df_baseline.mean()

V
a       0.136355
c       0.163125
eps    71.619914
i       0.975681
k       0.207711
l       0.126937
r       0.233809
w       0.158989
y       0.256930
dtype: float64

Korrelációk:

In [9]:
sim_df_baseline.corr()

V,a,c,eps,i,k,l,r,w,y
V,,,,,,,,,
a,1.000000,0.792663,-0.009862,0.933948,0.421304,0.876850,0.671702,0.850024,0.986635
c,0.792663,1.000000,-0.008597,0.544833,0.829964,0.432107,0.146603,0.981336,0.862672
eps,-0.009862,-0.008597,1.000000,-0.008408,-0.006192,-0.007568,-0.009466,-0.009580,-0.009662
i,0.933948,0.544833,-0.008408,1.000000,0.069336,0.990761,0.884108,0.608284,0.894118
k,0.421304,0.829964,-0.006192,0.069336,1.000000,-0.066595,-0.368569,0.829061,0.485069
l,0.876850,0.432107,-0.007568,0.990761,-0.066595,1.000000,0.934378,0.495693,0.828343
r,0.671702,0.146603,-0.009466,0.884108,-0.368569,0.934378,1.000000,0.200512,0.611540
w,0.850024,0.981336,-0.009580,0.608284,0.829061,0.495693,0.200512,1.000000,0.890975
y,0.986635,0.862672,-0.009662,0.894118,0.485069,0.828343,0.611540,0.890975,1.000000


**Kérdések:**

  - Mennyire adja vissza jól a modell a megfigyelt korrelációkat?
  - Mi a helyzet a munka szórásával?
  - Mi a helyzet a munka prociklikusságával (munka-GDP korreláció)?
  - Mi lehet ez utóbbinak az oka?

## 2. feladat: Oszthatatlan munka